In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql.functions import col
from pyspark.sql.types import TimestampType

spark = SparkSession.builder.appName("Project2").getOrCreate()

spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

output_dir = "output/"
task2_output_total = output_dir + "task2/task2_total.csv"
task2_output_avgsales = output_dir + "task2/task2_avgsales.csv"
task2_output_season = output_dir + "task2/task2_season.csv"
task3_output = output_dir + "task3/task3.csv"
task4_output = output_dir + "task4/task4.csv"

checkpoint_dir = "checkpoint/task5/"
task5_output = output_dir + "task5.csv"

24/11/12 23:57:36 WARN Utils: Your hostname, codespaces-38b548 resolves to a loopback address: 127.0.0.1; using 10.0.2.61 instead (on interface eth0)
24/11/12 23:57:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/12 23:57:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
train_df = spark.read.json("train.json")
reviwes_df = spark.read.json("test.json")
train_df.show()
#reviwes_df = reviwes_df.withColumn("helpful", F.col("helpful").cast("double"))
#reviwes_df.printSchema()
#print(reviwes_df.describe())
# Check if any rows have nulls in the 'helpful' column after casting
#invalid_rows = reviwes_df.filter(F.col("sentence").isNull())
# Show rows that could not be cast to double
#invalid_rows.show()

+----------+-------+--------------------+--------------------+--------------------+
|      asin|helpful|      main_image_url|       product_title|            sentence|
+----------+-------+--------------------+--------------------+--------------------+
|B000AO3L84|    1.7|http://ecx.images...|Canon 430EX Speed...|this flash is a s...|
|B001SEQPGK|    1.3|http://ecx.images...|Sony Cyber-shot D...|The pictures were...|
|0553386697|    1.9|http://ecx.images...|The Whole-Brain C...|A very good resou...|
|B006SUWZH2|   0.25|http://ecx.images...|Memorex Portable ...|We have it in a c...|
|B000W7F5SS|    0.9|http://ecx.images...|Harry Potter and ...|Again the makers ...|
|B000AO3L84|    2.0|http://ecx.images...|Canon 430EX Speed...|This flash is a g...|
|B00081NX5U|   0.73|http://ecx.images...|iPod Detachable R...|So I've had these...|
|B00000F1D3|    0.9|http://ecx.images...|             Believe|they're cd's or t...|
|B00000FCBH|    1.3|http://ecx.images...|  2Pac Greatest Hits|he proved that

In [3]:
reviwes_df.show()

+----------+-------+--------------------+--------------------+--------------------+
|      asin|helpful|      main_image_url|       product_title|            sentence|
+----------+-------+--------------------+--------------------+--------------------+
|B00VG90446|   1.07|http://ecx.images...|Flexion KS-902 Ki...|so it stays in pl...|
|B001196MG0|   1.33|http://ecx.images...|Savage 107X12-1 S...|Love this seamles...|
|B00081NX5U|   1.17|http://ecx.images...|iPod Detachable R...|very happy with m...|
|B003HC9JIW|    1.6|http://ecx.images...|Start! Walking At...|Even for someone ...|
|B00C30FCUI|   1.49|http://ecx.images...|Symphonized NRG P...|, those have alwa...|
|B001196MG0|   1.47|http://ecx.images...|Savage 107X12-1 S...|but after a year ...|
|B00AR1G3FS|   1.24|http://ecx.images...|Farewell Live Fro...|While not quite a...|
|B007R3AZNK|   0.67|http://ecx.images...|Driving Towards T...|Until now, Sloe G...|
|0761165975|    1.0|http://ecx.images...|The Wedding Plann...|I considered t

In [4]:
online_retail_df = spark.read.csv("Online-Retail.csv", inferSchema=True, header=True)

In [5]:
online_retail_df.select([F.count(F.when(col(c).isNull(), c)).alias(c) for c in online_retail_df.columns]).show()

# Drop rows with nulls in important columns like CustomerID, Description, etc.
online_retail_df = online_retail_df.dropna(subset=["CustomerID", "Description", "InvoiceDate", "Quantity", "UnitPrice"])
#online_retail_df.select([F.count(F.when(col(c).isNull(), c)).alias(c) for c in online_retail_df.columns]).show()

+---------+---------+-----------+--------+-----------+---------+----------+-------+
|InvoiceNo|StockCode|Description|Quantity|InvoiceDate|UnitPrice|CustomerID|Country|
+---------+---------+-----------+--------+-----------+---------+----------+-------+
|        0|        0|       1454|       0|          0|        0|    135080|      0|
+---------+---------+-----------+--------+-----------+---------+----------+-------+



In [6]:
#drop duplicates
online_retail_df = online_retail_df.dropDuplicates()

In [7]:
#cast the invoice date to timestamp
online_retail_df = online_retail_df.withColumn("InvoiceDate", F.to_timestamp("InvoiceDate", "M/d/yyyy H:mm"))
online_retail_df = online_retail_df.withColumn("InvoiceDate", F.col("InvoiceDate").cast(TimestampType()))

In [8]:
#check for outliers
online_retail_df.filter((col("Quantity") < 0) | (col("UnitPrice") < 0)).show()

# Drop rows where Quantity or UnitPrice are negative (common outlier check)
online_retail_df = online_retail_df.filter((F.col("Quantity") >= 0) & (F.col("UnitPrice") >= 0))
#online_retail_df.filter((col("Quantity") < 0) | (col("UnitPrice") < 0)).show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|  C538350|   85099F|JUMBO BAG STRAWBERRY|      -2|2010-12-10 15:01:00|     1.65|     13798|United Kingdom|
|  C538375|    22220|CAKE STAND LOVEBI...|      -1|2010-12-12 11:19:00|     9.95|     17126|United Kingdom|
|  C539726|    22791|T-LIGHT GLASS FLU...|     -10|2010-12-21 14:24:00|     1.25|     17007|United Kingdom|
|  C540307|    22084|PAPER CHAIN KIT E...|     -36|2011-01-06 12:58:00|     2.95|     15823|United Kingdom|
|  C542138|    20866|BLUE ROSE FABRIC ...|    -120|2011-01-25 17:21:00|     1.06|     17368|United Kingdom|
|  C543347|    22629| SPACEBOY LUNCH BOX |      -1|2011-02-07 12:44:00|     1.95|     12472|       Germany|
|  C544830|    22059|CERAMIC

In [9]:
#standardize for consistent description characters
online_retail_df = online_retail_df.withColumn("Description", F.upper(F.col("Description")))

#handle special cases in case for better performance for MLlib
#online_retail_df = online_retail_df.filter(~F.col("InvoiceNo").startswith("C"))

In [10]:
online_retail_df.show()

24/11/12 23:57:51 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536367|    22745|POPPY'S PLAYHOUSE...|       6|2010-12-01 08:34:00|      2.1|     13047|United Kingdom|
|   536368|    22960|JAM MAKING SET WI...|       6|2010-12-01 08:34:00|     4.25|     13047|United Kingdom|
|   536388|    22915|ASSORTED BOTTLE T...|      12|2010-12-01 09:59:00|     0.42|     16250|United Kingdom|
|   536401|    21464|DISCO BALL ROTATO...|       1|2010-12-01 11:21:00|     4.25|     15862|United Kingdom|
|   536412|    22569|FELTCRAFT CUSHION...|       2|2010-12-01 11:49:00|     3.75|     17920|United Kingdom|
|   536425|    22645|CERAMIC HEART FAI...|      12|2010-12-01 12:08:00|     1.45|     13758|United Kingdom|
|   536488|    22376|AIRLINE

In [11]:
#-----------------------------------
#Task 2: Sales Data Aggregation and Feature Engineering
#-----------------------------------
#total sales per product per month
#get the month and year from invoicedate
online_retail_df = online_retail_df.withColumn("Month", F.month("InvoiceDate")).withColumn("Year", F.year("InvoiceDate"))

#make a revenue column, total quantity * price
online_retail_df = online_retail_df.withColumn("Revenue", col("Quantity") * col("UnitPrice"))

#total sales per product and month. calculated by summing total quantity * price
total_sales_df = online_retail_df.groupBy("StockCode", "Month", "Year").agg(F.sum("Revenue").alias("TotalSales"))
total_sales_df.show()

+---------+-----+----+------------------+
|StockCode|Month|Year|        TotalSales|
+---------+-----+----+------------------+
|    22963|    1|2011|254.98000000000002|
|    22720|    3|2011|            3614.7|
|    23165|    6|2011| 789.6000000000001|
|    22549|    1|2011|              87.0|
|    22804|    3|2011| 638.1500000000001|
|    22055|    3|2011|345.65000000000003|
|    22629|    4|2011| 922.6499999999997|
|    37500|    3|2011|198.00000000000003|
|    22407|    4|2011|             18.75|
|    23140|    4|2011|            152.25|
|    22090|    7|2011|2018.8000000000002|
|    22899|    1|2011|             292.1|
|    21217|    4|2011| 708.1999999999998|
|    23156|    4|2011|350.84000000000003|
|    22328|    6|2011|1557.4500000000003|
|    21930|    6|2011|1285.7999999999995|
|    20679|    6|2011|243.95000000000005|
|   85035A|    7|2011|              51.0|
|    22604|    8|2011|             53.55|
|    21747|    1|2011|              35.0|
+---------+-----+----+------------

In [12]:
#average revenue per customer
#get average revnue for each customer id
avg_revnue_df = online_retail_df.groupBy("CustomerID").agg(F.avg("Revenue").alias("AverageRevenue")).orderBy(F.desc("AverageRevenue"))
avg_revnue_df.show()

+----------+------------------+
|CustomerID|    AverageRevenue|
+----------+------------------+
|     12346|           77183.6|
|     16446|           56157.5|
|     15098|           13305.5|
|     15749|           4453.43|
|     15195|            3861.0|
|     13135|            3096.0|
|     17846|            2033.1|
|     18087|2027.8599999999997|
|     16532|            1687.2|
|     16000|1377.0777777777778|
|     16754|            1001.2|
|     12755| 952.9874999999998|
|     18133| 931.4999999999999|
|     12798| 872.1299999999999|
|     17949| 835.8640000000001|
|     17553|             743.8|
|     15299| 643.8585714285715|
|     16308|             640.0|
|     16986|             624.4|
|     18080|            615.75|
+----------+------------------+
only showing top 20 rows



In [13]:
#seasonal patterns for top selling products
#get products by stock code and their highest total revnue as sum
top_products_df = online_retail_df.groupBy("StockCode").agg(F.sum("Revenue").alias("TotalRevenue")).orderBy(F.desc("TotalRevenue"))
    
#join with df to get montly data, group by product and month for each of their total revenue
seasonal_pattern = online_retail_df.join(top_products_df.limit(10), "StockCode").groupBy("StockCode", "Month").agg(F.sum("Revenue").alias("MonthlySales"))
seasonal_pattern.show()

+---------+-----+------------------+
|StockCode|Month|      MonthlySales|
+---------+-----+------------------+
|   85123A|   10| 4552.740000000001|
|     POST|    1|            5109.0|
|   85099B|   10| 9763.060000000005|
|    84879|    2| 2620.909999999999|
|     POST|   10|            7993.5|
|     POST|   12|            7190.0|
|    47566|   11|3715.7099999999996|
|   85123A|    1|14962.049999999996|
|   85123A|    3| 5262.499999999999|
|    84879|    9| 4866.639999999997|
|    47566|    9|            4387.0|
|     POST|    6|           5510.02|
|        M|   10|          21183.63|
|        M|   12|            666.27|
|    84879|    3|3997.9799999999987|
|   85123A|    5|10700.419999999995|
|    47566|    3| 6129.150000000001|
|    23166|    7|            826.94|
|     POST|    9|            5894.5|
|    23166|   10|            283.67|
+---------+-----+------------------+
only showing top 20 rows



In [14]:
#customer liftime value: total revenue per customer
clv_df = online_retail_df.groupBy("CustomerID").agg(F.sum("Revenue").alias("CustomerLifetimeValue"))
clv_df.show()

+----------+---------------------+
|CustomerID|CustomerLifetimeValue|
+----------+---------------------+
|     14570|               218.06|
|     13285|              2709.12|
|     16503|   1431.9300000000003|
|     13623|               727.74|
|     15727|    5159.060000000001|
|     18024|               389.78|
|     17389|             31833.68|
|     14450|    483.2500000000001|
|     17420|    598.8299999999999|
|     16339|    94.05000000000001|
|     15447|   155.17000000000002|
|     16386|   302.57000000000005|
|     12940|    899.6900000000002|
|     17679|   1992.1100000000001|
|     15957|               428.89|
|     15619|   336.40000000000003|
|     16574|               451.44|
|     15790|   218.74999999999994|
|     16861|   173.76000000000002|
|     13832|   52.199999999999996|
+----------+---------------------+
only showing top 20 rows



In [15]:
#product popularity: counted by unique transaction made under each stock code
product_popularity = online_retail_df.groupBy("StockCode").agg(F.countDistinct("InvoiceNo").alias("PopularityScore"))
product_popularity.show()

+---------+---------------+
|StockCode|PopularityScore|
+---------+---------------+
|    21889|            449|
|    21259|            237|
|    22728|            613|
|    21452|            133|
|    22121|            114|
|    21894|             71|
|    90143|              7|
|    22596|            234|
|    84881|              5|
|    21248|             52|
|    22254|             41|
|    21249|             79|
|    23318|            329|
|   90026D|              3|
|    90022|              4|
|    21331|              7|
|    23459|             19|
|    20868|             31|
|   90210B|              6|
|    23843|              1|
+---------+---------------+
only showing top 20 rows



In [16]:
#seasonal trends:
#make season column based on month
online_retail_df = online_retail_df.withColumn("Season",
                    F.when(col("Month").isin(12, 1, 2), "Winter") #if month is in one of these numbers
                    .when(col("Month").isin(3, 4, 5), "Spring")
                    .when(col("Month").isin(6, 7, 8), "Summer")
                    .when(col("Month").isin(9, 10, 11), "Fall")
                    )
    
#total revenue of each product and season
seasonal_trends = online_retail_df.groupBy("StockCode", "Season").agg(F.sum("Revenue").alias("SeasonalSales"))
seasonal_trends.show()

+---------+------+------------------+
|StockCode|Season|     SeasonalSales|
+---------+------+------------------+
|    21110|Winter|            1349.3|
|    22668|Winter|            1411.7|
|    23203|Summer|15717.460000000015|
|    21213|Summer|1750.9500000000003|
|    21371|Summer|             129.0|
|    22728|Summer|           4546.26|
|    22966|Winter|2459.8599999999997|
|   90002D|Winter|              30.0|
|    22149|Spring|            2487.7|
|    22661|Summer|1333.0500000000002|
|    22089|Summer| 926.7500000000002|
|    37446|Winter| 659.5999999999999|
|    22252|Spring|             97.94|
|    22760|Winter|             849.9|
|   84558A|Winter|268.45000000000005|
|    22421|Winter|            159.48|
|    22301|Winter|1107.4499999999998|
|    22537|Spring|            190.26|
|    23147|Summer|1035.2999999999995|
|    16219|Summer|197.99999999999997|
+---------+------+------------------+
only showing top 20 rows



In [17]:
#write the aggregated data to the directory as csv files
try:
    total_sales_df.write.csv(task2_output_total, header=True)
    avg_revnue_df.write.csv(task2_output_avgsales, header=True)
    seasonal_pattern.write.csv(task2_output_season, header=True)
except ValueError as e:
    print(f"error {e}")